# Running DenseNet on CIFAR-10 Datasets


To make this DenseNet memory efficient, I tried creating wide DenseNet by applying few methods like:
1. making the network shallow (set the depth to be smaller) 
2. making the network wider (set the growth rate k which is num_filter in this case to be larger).

Here, I am also using image augmentation which makes the model accuracy better for unseen data.

# Parameters used:

* No. of epochs: 35
* L: 14
* No. of filters (K): 64

# Accuracy:

10000/10000 [==============================] - 57s 6ms/step
* Test loss: 0.36021954385042193
* Test accuracy: 0.8849



In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 35
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 7s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    ''' Apply BatchNorm, Relu, 3x3 Conv2D and dropout
    Args:
        input: Input keras tensor
        num_filter: number of filters
        bottleneck: add bottleneck block
        dropout_rate: dropout rate
    Returns: keras tensor with batch_norm, relu and convolution2d added.
    '''
    
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    ''' Apply BatchNorm, Relu, 1x1 Conv2D and dropout
    Args:
        input: Input keras tensor
        num_filter: number of filters
        dropout_rate: dropout rate
    Returns: keras tensor with batch_norm, relu, convolution2d and MaxPooling added.
    '''
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = MaxPooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    ''' Apply Relu, flatten and create a dense layer
    Args:
        input: Input keras tensor
        
    Returns: keras tensor with relu, flatten and softmax added.
    '''
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 64
dropout_rate = 0.2
l = 14 
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1728        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

conv2d_59 (Conv2D)              (None, 4, 4, 32)     119808      activation_58[0][0]              
__________________________________________________________________________________________________
dropout_58 (Dropout)            (None, 4, 4, 32)     0           conv2d_59[0][0]                  
__________________________________________________________________________________________________
concatenate_55 (Concatenate)    (None, 4, 4, 448)    0           concatenate_54[0][0]             
                                                                 dropout_58[0][0]                 
__________________________________________________________________________________________________
batch_normalization_59 (BatchNo (None, 4, 4, 448)    1792        concatenate_55[0][0]             
__________________________________________________________________________________________________
activation_59 (Activation)      (None, 4, 4, 448)    0           batch_normalization_59[0][0]     
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# Adding Data Augmentation

In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(zoom_range=0.2, 
                             horizontal_flip=True)




In [13]:
'''model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
'''

# Train the model
model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size),
                    steps_per_epoch = 400,
                    epochs = epochs,validation_data = (x_test, y_test),
                    verbose=1)



Epoch 1/35
400/400 [==============================] - 679s 2s/step - loss: 1.5556 - acc: 0.4312 - val_loss: 2.1032 - val_acc: 0.3487
Epoch 2/35
279/400 [===================>..........] - ETA: 3:02 - loss: 1.1271 - acc: 0.5948400/400 [==============================] - 652s 2s/step - loss: 1.0777 - acc: 0.6131 - val_loss: 1.4156 - val_acc: 0.5593
Epoch 3/35
400/400 [==============================] - 652s 2s/step - loss: 0.8537 - acc: 0.6949 - val_loss: 0.9348 - val_acc: 0.6846
Epoch 4/35
 11/400 [..............................] - ETA: 9:45 - loss: 0.7871 - acc: 0.7216400/400 [==============================] - 652s 2s/step - loss: 0.7374 - acc: 0.7402 - val_loss: 0.7957 - val_acc: 0.7287
Epoch 5/35
319/400 [======================>.......] - ETA: 2:02 - loss: 0.6757 - acc: 0.7634400/400 [==============================] - 652s 2s/step - loss: 0.6672 - acc: 0.7668 - val_loss: 0.6956 - val_acc: 0.7658
Epoch 6/35
400/400 [==============================] - 651s 2s/step - loss: 0.6164 - acc: 0.7

In [14]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 57s 6ms/step
Test loss: 0.36021954385042193
Test accuracy: 0.8849


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

In [0]:
from google.colab import files

files.download('DNST_model.h5')